![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [216]:
import pandas as pd
import numpy as np

# Start coding here...

In [217]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [219]:
client = df[['client_id','age','job','marital','education','credit_default','mortgage']]

In [220]:
campaign = df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome', 'campaign_outcome']]

In [221]:
campaign['last_contact_date'] = df['month'] + ' ' + df['day'].astype(str)

In [222]:
economics = df[['client_id','cons_price_idx','euribor_three_months']]

In [223]:
client['education']= client['education'].str.replace('.','_')
client['education']= client['education'].replace('unknown',np.NaN)
client['job'] = client['job'].str.replace('.','_')


client['education'].unique()

array(['basic_4y', 'high_school', 'basic_6y', 'basic_9y',
       'professional_course', nan, 'university_degree', 'illiterate'],
      dtype=object)

In [224]:
campaign['campaign_outcome']= campaign['campaign_outcome'].map({'yes':1,'no':0})

In [225]:
campaign['previous_outcome'] = campaign['previous_outcome'].map({'success':1,'nonexistent':0,'failure':0})

In [226]:
client['mortgage']=client['mortgage'].map({'yes':1,'no':0, 'unknown':0})
client['credit_default']=client['credit_default'].map({'yes':1,'no':0, 'unknown':0})

client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,0,0
1,1,57,services,married,high_school,0,0
2,2,37,services,married,high_school,0,1
3,3,40,admin_,married,basic_6y,0,0
4,4,56,services,married,high_school,0,0


In [227]:
campaign['year'] = 2022

In [228]:
campaign['last_contact_date'] = campaign['last_contact_date'] + ' ' + campaign['year'].astype(str)

In [229]:
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], infer_datetime_format=True,errors = 'coerce')

campaign['last_contact_date'] = campaign['last_contact_date'].dt.strftime('%d-%m-%y')

campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date,year
0,0,1,261,0,0,0,13-05-22,2022
1,1,1,149,0,0,0,19-05-22,2022
2,2,1,226,0,0,0,23-05-22,2022
3,3,1,151,0,0,0,27-05-22,2022
4,4,1,307,0,0,0,03-05-22,2022


In [230]:
campaign['campaign_outcome']=campaign['campaign_outcome'].astype(bool)
campaign['previous_outcome']=campaign['previous_outcome'].astype(bool)
client['credit_default']=client['credit_default'].astype(bool)
client['mortgage']=client['mortgage'].astype(bool)

In [231]:
print(client.dtypes)
print(campaign.dtypes)

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object
client_id                      int64
number_contacts                int64
contact_duration               int64
previous_campaign_contacts     int64
previous_outcome                bool
campaign_outcome                bool
last_contact_date             object
year                           int64
dtype: object


In [232]:
campaign.drop('year',axis='columns',inplace=True)
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,13-05-22
1,1,1,149,0,False,False,19-05-22
2,2,1,226,0,False,False,23-05-22
3,3,1,151,0,False,False,27-05-22
4,4,1,307,0,False,False,03-05-22


In [233]:
campaign.to_csv('campaign.csv',index = False)
client.to_csv('client.csv', index = False)
economics.to_csv('economics.csv', index = False)